In [39]:
import pandas as pd
import numpy as np
import operator
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.data import load
from nltk.stem import SnowballStemmer
from string import punctuation
import re

#LEER EL DATASET

df=pd.read_csv('Dataset.csv')
#L=np.asarray(df)
#print(L)

data = pd.read_csv("Dataset.csv", sep=",")
##########################################
punctuation = list(string.punctuation)
stemmer = SnowballStemmer("spanish")
stop = stopwords.words('spanish') + punctuation
vc = TfidfVectorizer()
#############Documentos##################
Tweet = data[["Tweet"]]
T=np.asarray(Tweet)
bigNumOfWords = []
bigBagOfWords = []
unique = [' ']
bigTf = {}
bigTfidf= []
########################################
emoticons_str = r"""
    (?:
        [:=;] # :(
        [oO\-]? # No se
        [D\)\]\(\]/\\OpP] # 
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
##############Metodos####################



def generate_ngrams(s, n):
    # Convert to lowercases
    s = s.lower()
    # Romper la oración en el tokens
    tokens_first = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE )
    tokens = tokens_first.findall(s)
        
    # eliminar tokens vacíos, links, menciones, stopwords y hastags
    tokens = [token for token in tokens if (token != "" ) and (token not in stop) and (not token.startswith(('#', '@', 'https:', 'http:')))]
    #lematizar
    tokens = [stemmer.stem(token) for token in tokens]
    # funcion zip genera n-grams
    # Concatena los tokens en los ngrams y los retorna
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]

def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        if bagOfWordsCount != 0:
            tfDict[word] = count / float(bagOfWordsCount)
        else:
            tfDict[word] = 0

    return tfDict


def computeIDF(documents):
    import math
    N = len(documents)
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    
    for document in documents:
        for word, val in document.items():
            
            if val > 0:
                idfDict[word] += 1
    for word, val in idfDict.items():
        if val != 0:
            idfDict[word] = math.log(N / float(val))
        else:
            idfDict[word] = 0
    return idfDict

def computeTFIDF(tfBagOfWords, idfs):
    tfidf = {}
    for word, val in tfBagOfWords.items():
        tfidf[word] = val * idfs[word]
    return tfidf

#########################################

##########Preprocesamiento##########
n=2
count = 0
count2 = 0
for tweet in T:
    bagOfWords = generate_ngrams(str(tweet), n)
    unique = set(unique).union(set(bagOfWords))
    bigBagOfWords.insert(0, bagOfWords)
############Contar Palabras######################
for tweet in bigBagOfWords:
    numOfWords = dict.fromkeys(unique, 0)
    for word in tweet:
        numOfWords[word] += 1
    bigNumOfWords.append(numOfWords)

count = 0
##################TF##########################
for tweet in T:
    tf = computeTF(bigNumOfWords[count], bigBagOfWords[count])
    bigTf[count] = tf
    count += 1
#####################IDF###################
idfs = computeIDF(bigNumOfWords)
###################TFIDF###################

for tweet in T:
    tfidf = computeTFIDF(bigTf[count2], idfs)
    bigTfidf.append(tfidf)
    count2 += 1
#################results############################33
df = pd.DataFrame(bigTfidf)
df.to_csv (r'TFIDF.csv', index = True, header=True)

In [33]:
print(str(T[122]))
print(generate_ngrams(str(T[122]), 1))

['@JEP_Colombia Partida de malandros es lo que son.']
['part', 'malandr']


In [38]:
print(bigNumOfWords[0])

{'demuestr colombian': 0, 'fundamental pued': 0, 'campesin habri': 0, 'obten benefici': 0, 'poc fond': 0, 'despu ventaj': 0, 'conden aun': 0, 'mal narr': 0, 'jep mega-contrat': 0, 'verd verd': 0, 'elig defend': 0, 'construccion edifici': 0, 'diferent social': 0, 'quier si': 0, 'verdader codig': 0, 'desconoc acuerd': 0, 'amarres pantalon': 0, 'dan sicolog': 0, 'jep actor': 0, 'espald culp': 0, 'muert atent': 0, 'pas gustav': 0, 'apto jep': 0, 'eleccion eliminari': 0, 'pues servici': 0, 'acuerd dpaz': 0, 'pierd tirani': 0, 'deten masacr': 0, 'millon mensual': 1, 'milit air': 0, 'puebl quier': 0, 'tem dialog': 0, 'vuelt edifici': 0, 'crimen farc': 0, 'segur polici': 0, 'cabal culpabl': 0, 'compromet jep': 0, 'cort orej': 0, 'cartuch magistral': 0, 'gremi recuerd': 0, 'mamert llev': 0, 'burr dic': 0, 'benefici deplor': 0, 'alcald mucipi': 0, 'pit penal': 0, 'jep trat': 0, 'estan sen': 0, 'cataliz impun': 0, 'narcojustici cartel': 0, 'petrism esfuerzat': 0, 'inici sistemat': 0, 'narcoterror

In [6]:
print(len(bigTf))

2130


In [40]:
print(bigTfidf[0])

{'demuestr colombian': 0.0, 'fundamental pued': 0.0, 'campesin habri': 0.0, 'obten benefici': 0.0, 'poc fond': 0.0, 'despu ventaj': 0.0, 'conden aun': 0.0, 'mal narr': 0.0, 'jep mega-contrat': 0.0, 'verd verd': 0.0, 'elig defend': 0.0, 'construccion edifici': 0.0, 'diferent social': 0.0, 'quier si': 0.0, 'verdader codig': 0.0, 'desconoc acuerd': 0.0, 'amarres pantalon': 0.0, 'dan sicolog': 0.0, 'jep actor': 0.0, 'espald culp': 0.0, 'muert atent': 0.0, 'pas gustav': 0.0, 'apto jep': 0.0, 'eleccion eliminari': 0.0, 'pues servici': 0.0, 'acuerd dpaz': 0.0, 'pierd tirani': 0.0, 'deten masacr': 0.0, 'millon mensual': 0.3126316652397791, 'milit air': 0.0, 'puebl quier': 0.0, 'tem dialog': 0.0, 'vuelt edifici': 0.0, 'crimen farc': 0.0, 'segur polici': 0.0, 'cabal culpabl': 0.0, 'compromet jep': 0.0, 'cort orej': 0.0, 'cartuch magistral': 0.0, 'gremi recuerd': 0.0, 'mamert llev': 0.0, 'burr dic': 0.0, 'benefici deplor': 0.0, 'alcald mucipi': 0.0, 'pit penal': 0.0, 'jep trat': 0.0, 'estan sen':

In [41]:
print(df)

      demuestr colombian  fundamental pued  campesin habri  obten benefici  \
0                    0.0               0.0             0.0             0.0   
1                    0.0               0.0             0.0             0.0   
2                    0.0               0.0             0.0             0.0   
3                    0.0               0.0             0.0             0.0   
4                    0.0               0.0             0.0             0.0   
...                  ...               ...             ...             ...   
2125                 0.0               0.0             0.0             0.0   
2126                 0.0               0.0             0.0             0.0   
2127                 0.0               0.0             0.0             0.0   
2128                 0.0               0.0             0.0             0.0   
2129                 0.0               0.0             0.0             0.0   

      poc fond  despu ventaj  conden aun  mal narr  jep mega-co